In [1]:
import os
import pandas as pd
import sys
import time

from PIL import Image
from sqlalchemy import create_engine, func
from sqlalchemy.orm import sessionmaker

sys.path.append('../..')
sys.path.append('../../../ajna_docs/commons')


In [2]:
from ajna_commons.flask.conf import SQL_URI
from ajna_commons.utils.images import mongo_image
from virasana.db import mongodb as db


2021-03-05 11:02:01,515 ajna         INFO     Configuração de log efetuada


Fazendo log de erros e alertas no arquivo  ../../../ajna_docs/commons\ajna_commons\flask\error.log
Fazendo log de atividade no arquivo  ../../../ajna_docs/commons\ajna_commons\flask\access.log


In [23]:
%load_ext autoreload 
%autoreload 2
from virasana.integracao.conformidade_alchemy import Conformidade
from virasana.scripts.conformidadeupdate import completa_conformidade, preenche_isocode, preenche_bbox
engine = create_engine(SQL_URI)
# completa_conformidade(db, engine, 200)
# preenche_isocode(db, engine, 200)
preenche_bbox(db, engine, 20)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2
2
1
2
1
2
2
1
1
1
2
1
1
1
2
1
1
1
1
2


2021-03-05 11:22:27,353 ajna         INFO     20 bbox preenchidos em 5.685836315155029 segundos.0.2842918157577515 por registro


# Checagens de conformidade

In [6]:
Session = sessionmaker(bind=engine)
session = Session()

In [7]:
rs = session.execute('SELECT count(*) FROM dbmercante.ajna_conformidade;')

In [8]:
rs.scalar()

124896

In [30]:
SQL_CONFORMIDADE = '''
SELECT cod_recinto as Recinto, count(*) as "Qtde de imagens",
 avg(height) as "Linhas",
 avg(width) as "Colunas",
 avg(ratio) as "Relação largura/altura",
 SUM(if(height < 700, 1, 0)) / count(*) * 100 as "% Tamanho pequeno",
 SUM(if(ratio < 1.9, 1, 0)) / count(ratio) * 100 as "% relação abaixo 1.9",
 avg(laplacian) as "Índice de nitidez ou ruído",
 avg(bbox_score) as "% Confiança da VC",
 sum(bbox_classe>=3) / count(*) as "% VC não encontra CC"
 from ajna_conformidade 
 group by cod_recinto
 '''
df = pd.read_sql(SQL_CONFORMIDADE, engine)

In [31]:
df

,Recinto,Qtde de imagens,Linhas,Colunas,Relação largura/altura,% Tamanho pequeno,% relação abaixo 1.9,Índice de nitidez ou ruído,% Confiança da VC,% VC não encontra CC
0,BANDEIRANTES,1780,799.3517,1919.5792,2.401938,0.0000,22.6404,113.5556,NaN,NaN
1,BTP1,28219,799.0278,2125.6738,2.660360,0.0000,4.8194,89.3529,NaN,NaN
2,BTP2,13400,799.0275,1860.7266,2.328790,0.0000,14.9030,112.0861,1.00,0.0
3,CRAGEA,640,879.9313,1912.4156,2.173406,0.0000,10.4688,NaN,NaN,NaN
4,DEICMAR,4238,799.0647,1343.8495,1.681654,0.0000,76.7343,131.3846,NaN,NaN
5,ECOPORTO,5238,798.6682,2133.7892,2.672073,0.0573,3.3219,84.3611,NaN,NaN
6,EMBRAPORT,17206,799.3174,3687.2964,4.613713,0.0000,0.1802,791.0000,0.98,0.0
7,EUDMARCO,2234,234.3505,882.7543,3.929624,92.8380,0.0000,1359.0000,NaN,NaN
8,LOCALFRIO,5554,799.0013,1760.9301,2.204316,0.0000,3.1329,112.5862,1.00,0.0
9,MARIMEX,5836,799.0281,2037.5778,2.550190,0.0000,10.4352,92.0526,1.00,0.0
